<a href="https://colab.research.google.com/github/MS1997/Autoencoders-with-skip-connections/blob/master/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 # installing kaggle library to import the data directly into Colab notebook
 ! pip install -q kaggle

In [0]:
# select file containing API key from local system
from google.colab import files
uploaded = files.upload()

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle datasets download -d hsankesara/flickr-image-dataset

In [0]:
# unzipping the files
! unzip flickr-image-dataset.zip -d flickr-image-dataset

In [0]:
import os
import shutil
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Conv2D, Conv2DTranspose, Input, MaxPooling2D, Add, BatchNormalization, LeakyReLU, Reshape, Flatten, Dense
from keras.optimizers import Adam
import numpy as np
from random import sample 
from keras.models import Model
import matplotlib.pyplot as plt
import tensorflow as tf

In [0]:
os.remove('flickr-image-dataset.zip')
os.remove('flickr-image-dataset/flickr30k_images/results.csv')
list_i = os.listdir('flickr-image-dataset/flickr30k_images/flickr30k_images') #31785
len(list_i)

In [0]:
# making directory to store 10k images 
os.mkdir('flickr-image-dataset/selected')
os.mkdir('flickr-image-dataset/selected/train')

In [0]:
# let's choose 10k random images from the orginal 31k images
s_10k = sample(list_i,10000)
for i in s_10k:
  source = 'flickr-image-dataset/flickr30k_images/flickr30k_images/' + i
  destination = 'flickr-image-dataset/selected/train/' + i
  os.rename(source, destination)

In [0]:
shutil.rmtree('flickr-image-dataset/flickr30k_images/flickr30k_images') # removing orginal directory 

In [0]:
os.listdir('flickr-image-dataset/flickr30k_images/flickr30k_images/')

In [0]:
X_train=[]
# converting the train images to a tensor and standardizing the pixel values
for i in os.listdir('flickr-image-dataset/selected/train/'):
  image_path = 'flickr-image-dataset/selected/train/' + i
  img = load_img(image_path, target_size=(128, 128))
  x = img_to_array(img)
  x = x.astype('float32')/255.
  X_train.append(x)

In [0]:
X_train=np.array(X_train)
X_train.shape # (10000, 128, 128, 3)

In [0]:
# adding noise to the training images 
noise_factor = 0.2
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)  
X_train_noisy = np.clip(X_train_noisy, 0., 1.)

In [0]:
input_img = Input(shape=(128, 128, 3))
# Encoder
# strides of 2,2 works like max pooling and downsamples the image
y = Conv2D(32, (3, 3), padding='same',strides =(2,2))(input_img)
y = LeakyReLU()(y)
y = Conv2D(64, (3, 3), padding='same',strides =(2,2))(y)
y = LeakyReLU()(y)
y2 = Conv2D(128, (3, 3), padding='same',strides =(2,2))(y)
y = LeakyReLU()(y2)
y = Conv2D(256, (3, 3), padding='same',strides =(2,2))(y)
y = LeakyReLU()(y)
y1 = Conv2D(256, (3, 3), padding='same',strides =(2,2))(y)
y = LeakyReLU()(y1)
y = Conv2D(512, (3, 3), padding='same',strides =(2,2))(y)
y = LeakyReLU()(y)
y = Conv2D(1024, (3, 3), padding='same',strides =(2,2))(y)
y = LeakyReLU()(y)
vol = y.shape # shape of the final convolutional layer
x = Flatten()(y)
latent = Dense(128,activation='relu')(x) # bottleneck layer to control the information flow

In [0]:
# helper function 
def lrelu_bn(inputs):
 lrelu = LeakyReLU()(inputs)
 bn = BatchNormalization()(lrelu)
 return bn

In [0]:
# Decoder
y = Dense(np.prod(vol[1:]), activation='relu')(latent) # accepting the output from the bottleneck layer
y = Reshape((vol[1], vol[2], vol[3]))(y) 
y = Conv2DTranspose(1024, (3,3), padding='same')(y)
y = LeakyReLU()(y)
y = Conv2DTranspose(512, (3,3), padding='same',strides=(2,2))(y)
y = LeakyReLU()(y)
y = Conv2DTranspose(256, (3,3), padding='same',strides=(2,2))(y)
y= Add()([y1, y]) # remove to run model without skip connections
y = lrelu_bn(y)  # remove to run model without skip connections
y = Conv2DTranspose(256, (3,3), padding='same',strides=(2,2))(y)
y = LeakyReLU()(y) 
y = Conv2DTranspose(128, (3,3), padding='same',strides=(2,2))(y)   
y= Add()([y2, y]) # remove to run model without skip connections
y = lrelu_bn(y) # remove to run model without skip connections
y = Conv2DTranspose(64, (3,3), padding='same',strides=(2,2))(y)
y = LeakyReLU()(y)
y = Conv2DTranspose(32, (3,3), padding='same',strides=(2,2))(y)
y = LeakyReLU()(y)
y = Conv2DTranspose(3, (3,3), activation='sigmoid', padding='same',strides=(2,2))(y)

In [0]:
model_1 = Model(input_img,y)

In [0]:
model_1.summary()

In [0]:
model_1.compile(optimizer=Adam(0.001,beta_1=0.9), loss='binary_crossentropy',metrics=['accuracy'])
history_2 = model_1.fit(X_train_noisy,X_train,batch_size = 32,epochs = 200,validation_split=0.1)

In [0]:
# plot the train and validation loss
plt.plot(history_2.history['loss']) 
plt.plot(history_2.history['val_loss'])
plt.title('Comparison of Train & Validation loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left') 
plt.show()

In [0]:
# mounting the drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# saving model weights
model_1.save_weights("/content/gdrive/My Drive/model_1.h5") 

In [0]:
# preddiction on train images
preds= model_1.predict(X_train_noisy[0:4,:,:,:]) 

In [0]:
X_train_noisy.shape # (10000, 128, 128, 3)

In [0]:
plt.imshow(preds[1].reshape(128, 128,3)) # denoised image

In [0]:
plt.imshow(X_train[1].reshape(128, 128,3)) # orginal image

In [0]:
plt.imshow(X_train_noisy[2].reshape(128, 128,3)) # image with noise

In [0]:
model_1.load_weights('/content/gdrive/My Drive/model_2.h5') #load the previously saved weights 
# before running this the model defining block of code should be run

In [0]:
# loading the test image
from io import BytesIO
from google.colab import files
#uploading the image for 'content'
uploaded = files.upload()
content = Image.open(BytesIO(uploaded['columbia.jpg']))

In [0]:
# converting the test image to a tensor and adding noise 
img = load_img("columbia.jpg", target_size=(128, 128)) #
x = img_to_array(img)
x = x.astype('float32')/255.
noise_factor = 0.2
x_noisy = x + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x.shape)  
x_noisy = np.clip(x_noisy, 0., 1.)
x_noisy = np.expand_dims(x_noisy, axis=0)
x = np.expand_dims(x, axis=0)

In [0]:
plt.imshow(x.reshape(128, 128,3)) # orginal test image

In [0]:
plt.imshow(x_noisy.reshape(128, 128,3)) # test image with noise 

In [0]:
test_preds= model_1.predict(x_noisy)  # image reconstruction on test image with noise
plt.imshow(test_preds.reshape(128, 128,3))

In [0]:
! pip install keract # install keract package to see actiavtion outputs

In [0]:
import keract

In [0]:
act = keract.get_activations(model_1,x) # function to get activation outputs of the model

In [0]:
keract.display_activations(act) # function to display activation outputs of the model